In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [3]:
import pickle
data = pickle.load(open('gdrive/MyDrive/Spell Correction/Old_train.pkl','rb'))
print(len(data),len(set(i for i in data)))
import re
alphabet = '^[ _abcdefghijklmnopqrstuvwxyz0123456789áàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđ!\"\',\-\.:;?_\(\)]+$'

120000 119349


In [4]:
training_data=[]
for i in data:
  i=i.replace("\n",".")
  sentences=i.split(".")
  for j in sentences:
      if len(j.split()) > 2 and re.match(alphabet, j.lower()):
          training_data.append(j)
print(len(training_data))


119291


In [5]:
!pip install unidecode
import re
import numpy as np
from unidecode import unidecode
import pickle



letters=list("abcdefghijklmnopqrstuvwxyzáàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵđABCDEFGHIJKLMNOPQRSTUVWXYZÁÀẢÃẠÂẤẦẨẪẬĂẮẰẲẴẶÓÒỎÕỌÔỐỒỔỖỘƠỚỜỞỠỢÉÈẺẼẸÊẾỀỂỄỆÚÙỦŨỤƯỨỪỬỮỰÍÌỈĨỊÝỲỶỸỴĐ")
letters2=list("abcdefghijklmnopqrstuvwxyz")
telex={"ă":"aw","â":"aa","á":"as","à":"af","ả":"ar","ã":"ax","ạ":"aj","ắ":"aws","ổ":"oor","ỗ":"oox","ộ":"ooj","ơ":"ow",
"ằ":"awf","ẳ":"awr","ẵ":"awx","ặ":"awj","ó":"os","ò":"of","ỏ":"or","õ":"ox","ọ":"oj","ô":"oo","ố":"oos","ồ":"oof",
"ớ":"ows","ờ":"owf","ở":"owr","ỡ":"owx","ợ":"owj","é":"es","è":"ef","ẻ":"er","ẽ":"ex","ẹ":"ej","ê":"ee","ế":"ees","ề":"eef",
"ể":"eer","ễ":"eex","ệ":"eej","ú":"us","ù":"uf","ủ":"ur","ũ":"ux","ụ":"uj","ư":"uw","ứ":"uws","ừ":"uwf","ử":"uwr","ữ":"uwx",
"ự":"uwj","í":"is","ì":"if","ỉ":"ir","ị":"ij","ĩ":"ix","ý":"ys","ỳ":"yf","ỷ":"yr","ỵ":"yj","đ":"dd",
"Ă":"Aw","Â":"Aa","Á":"As","À":"Af","Ả":"Ar","Ã":"Ax","Ạ":"Aj","Ắ":"Aws","Ổ":"Oor","Ỗ":"Oox","Ộ":"Ooj","Ơ":"Ow",
"Ằ":"AWF","Ẳ":"Awr","Ẵ":"Awx","Ặ":"Awj","Ó":"Os","Ò":"Of","Ỏ":"Or","Õ":"Ox","Ọ":"Oj","Ô":"Oo","Ố":"Oos","Ồ":"Oof",
"Ớ":"Ows","Ờ":"Owf","Ở":"Owr","Ỡ":"Owx","Ợ":"Owj","É":"Es","È":"Ef","Ẻ":"Er","Ẽ":"Ex","Ẹ":"Ej","Ê":"Ee","Ế":"Ees","Ề":"Eef",
"Ể":"Eer","Ễ":"Eex","Ệ":"Eej","Ú":"Us","Ù":"Uf","Ủ":"Ur","Ũ":"Ux","Ụ":"Uj","Ư":"Uw","Ứ":"Uws","Ừ":"Uwf","Ử":"Uwr","Ữ":"Uwx",
"Ự":"Uwj","Í":"Is","Ì":"If","Ỉ":"Ir","Ị":"Ij","Ĩ":"Ix","Ý":"Ys","Ỳ":"Yf","Ỷ":"Yr","Ỵ":"Yj","Đ":"Dd"}
vni={"ă":"a8","â":"a6","á":"a1","à":"a2","ả":"a3","ã":"a4","ạ":"a5","ắ":"a81","ổ":"o63","ỗ":"o64","ộ":"o65","ơ":"o7",
"ằ":"a82","ẳ":"a83","ẵ":"a84","ặ":"a85","ó":"o1","ò":"o2","ỏ":"o3","õ":"o4","ọ":"o5","ô":"o6","ố":"o61","ồ":"o62",
"ớ":"o71","ờ":"o72","ở":"o73","ỡ":"o74","ợ":"o75","é":"e1","è":"e2","ẻ":"e3","ẽ":"e4","ẹ":"e5","ê":"e6","ế":"e61","ề":"e62",
"ể":"e63","ễ":"e64","ệ":"e65","ú":"u1","ù":"u2","ủ":"u3","ũ":"u4","ụ":"u5","ư":"u7","ứ":"u71","ừ":"u72","ử":"u73","ữ":"u74",
"ự":"u75","í":"i1","ì":"i2","ỉ":"i3","ị":"i5","ĩ":"i4","ý":"y1","ỳ":"y2","ỷ":"y3","ỵ":"y5","đ":"d6",
"Ă":"A8","Â":"A6","Á":"A1","À":"A2","Ả":"A3","Ã":"A4","Ạ":"A5","Ắ":"A81","Ổ":"O63","Ỗ":"O64","Ộ":"O65","Ơ":"O7",
"Ằ":"A82","Ẳ":"A83","Ẵ":"A84","Ặ":"A85","Ó":"O1","Ò":"O2","Ỏ":"O3","Õ":"O4","Ọ":"O5","Ô":"O6","Ố":"O61","Ồ":"O62",
"Ớ":"O71","Ờ":"O72","Ở":"O73","Ỡ":"O74","Ợ":"O75","É":"E1","È":"E2","Ẻ":"E3","Ẽ":"E4","Ẹ":"E5","Ê":"E6","Ế":"E61","Ề":"E62",
"Ể":"E63","Ễ":"E64","Ệ":"E65","Ú":"U1","Ù":"U2","Ủ":"U3","Ũ":"U4","Ụ":"U5","Ư":"U7","Ứ":"U71","Ừ":"U72","Ử":"U73","Ữ":"U74",
"Ự":"U75","Í":"I1","Ì":"I2","Ỉ":"I3","Ị":"I5","Ĩ":"I4","Ý":"Y1","Ỳ":"Y2","Ỷ":"Y3","Ỵ":"Y5","Đ":"D6"}
region={"ẻ":"ẽ","ẽ":"ẻ","ũ":"ủ","ủ":"ũ","ã":"ả","ả":"ã","ỏ":"õ","õ":"ỏ","i":"j"}
region2={"s":"x","l":"n","n":"l","x":"s","d":"gi","S":"X","L":"N","N":"L","X":"S","Gi":"D","D":"Gi"}
vowel=list("aeiouyáàảãạâấầẩẫậăắằẳẵặóòỏõọôốồổỗộơớờởỡợéèẻẽẹêếềểễệúùủũụưứừửữựíìỉĩịýỳỷỹỵ")


def add_noise(sentence, pivot1,pivot2):
    noisy_sentence = ""
    i = 0
    while i < len(sentence):
        if sentence[i] not in letters:
            noisy_sentence+=sentence[i]
        else:  
            random = np.random.uniform(0,1,1)[0]   
            if random < pivot1:
                noisy_sentence+=(sentence[i])
            elif random<pivot2:
                if sentence[i] in telex.keys() and sentence[i] in vni.keys() and sentence[i] in region.keys():
                    random2=np.random.uniform(0,1,1)[0]
                    if random2<=0.5:
                        noisy_sentence+=telex[sentence[i]]
                    elif random2<0.7:
                        noisy_sentence += vni[sentence[i]]
                    elif random2<0.9:
                        noisy_sentence+=region[sentence[i]]
                    elif random2<0.95 :
                        noisy_sentence+=unidecode(sentence[i])
                    else:
                        noisy_sentence+=sentence[i]
                elif sentence[i] in telex.keys():#Lỗi Telex
                    random3=np.random.uniform(0,1,1)[0]
                    if random3<=0.5:
                        noisy_sentence+=telex[sentence[i]]
                    elif random3<0.9 :
                        noisy_sentence+=unidecode(sentence[i])                        
                    else:
                        noisy_sentence+=sentence[i]
                elif sentence[i] in vni.keys():#Lỗi VNI
                    random4=np.random.uniform(0,1,1)[0]
                    if random4<=0.6:
                        noisy_sentence+=vni[sentence[i]]
                    elif random4<0.95 :
                        noisy_sentence+=unidecode(sentence[i])                        
                    else:
                        noisy_sentence+=sentence[i]
                elif sentence[i] in region.keys(): #Sai dấu vùng miền
                    random5=np.random.uniform(0,1,1)[0]
                    if random5<=0.5:
                        noisy_sentence+=region[sentence[i]]
                    elif random5<0.9:
                        noisy_sentence+=unidecode(sentence[i])                        
                    else:
                        noisy_sentence+=sentence[i]
                elif i<len(sentence)-1 :
                    if sentence[i] in region2.keys() and (i==0 or sentence[i-1] not in letters) and sentence[i+1] in vowel:
                        random6=np.random.uniform(0,1,1)[0]
                        if random6<=0.9:
                            noisy_sentence+=region2[sentence[i]]
                        else:
                            noisy_sentence+=sentence[i]
                    else:
                        noisy_sentence+=sentence[i]

            else:
                new_random = np.random.uniform(0,1,1)[0]
                if new_random <=0.33:
                    if i == (len(sentence) - 1):
                        continue
                    else:
                        noisy_sentence+=(sentence[i+1])
                        noisy_sentence+=(sentence[i])
                        i += 1
                elif new_random <= 0.66:
                    random_letter = np.random.choice(letters2, 1)[0]
                    noisy_sentence+=random_letter
                else:
                    pass
      
        i += 1
    return noisy_sentence



    #Lặp từ
    #Đảo từ
    #Lỗi telex
    #Lỗi VNI
    #Thiếu từ
    #Thiếu dấu
    

     |████████████████████████████████| 241 kB 6.5 MB/s 


In [6]:
def extract_phrases(text):
    return re.findall(r'\w[\w ]+', text)

import itertools
phrases = itertools.chain.from_iterable(extract_phrases(text) for text in training_data)
phrases = [p.strip() for p in phrases if len(p.split()) > 1]

print(len(phrases))

print(phrases[0:10])

119291
['đôi vợ chồng ngoài tuổi yokohama tìm đến kiến trúc sư với mong muốn có một ngôi nhà không có sự ngăn cách với bên ngoài mảnh đất xây nhà rộng hơn nằm trong khu vườn của bố mẹ người chồng xung quanh nhiều cây xanh ngoài nhu cầu sống gần gũi thiên nhiên gia chủ còn muốn bỏ đi những ranh giới ngăn cách họ với hàng xóm bên ngoài ngôi nhà ảnh shinkenchiku sha takumi ota jin hosoya để tạo ra không gian mở như gia chủ kiến trúc sư đưa ra thiết kế nhà ba tầng xuyên thấu bao quanh công trình là kính an toàn nhiều lớp từ mọi góc đôi vợ chồng cùng đứa con mới tám tháng tuổi có thể dễ dàng nhìn ra khu vườn cũng như các ngôi nhà gần đó đặc biệt kính dán nhiều lớp khi bị vỡ không văng mảnh nên đảm bảo an toàn cho người các thành viên thỏa sức ngắm cảnh bên ngoài qua lớp tường trong suốt ảnh shinkenchiku sha takumi ota jin hosoya bên trong ngôi nhà có bức tường gỗ chúng rộng mét tương đương chiều dài sải tay của chủ nhà được xếp quanh các trục tọa độ với mục đích tạo nên không gian nửa kín n

In [7]:
import re
import itertools
from nltk import ngrams
import string
import numpy as np
from tqdm import tqdm

def gen_ngrams(words, n=5):
    return ngrams(words.split(), n)
#print(gen_ngrams(phrases[0],5))
list_ngrams = []
for p in tqdm(phrases):
    if not re.match(alphabet, p.lower()):
        continue
    for ngr in gen_ngrams(p, 5):
        if len(" ".join(ngr)) < 30:
            list_ngrams.append(" ".join(ngr))
            




100%|██████████| 119291/119291 [00:59<00:00, 2007.70it/s]


In [8]:
del phrases
del training_data
list_ngrams = list((list_ngrams))
print(list_ngrams[0:10])
print(len(list_ngrams))

['đôi vợ chồng ngoài tuổi', 'vợ chồng ngoài tuổi yokohama', 'chồng ngoài tuổi yokohama tìm', 'ngoài tuổi yokohama tìm đến', 'tuổi yokohama tìm đến kiến', 'yokohama tìm đến kiến trúc', 'tìm đến kiến trúc sư', 'đến kiến trúc sư với', 'kiến trúc sư với mong', 'trúc sư với mong muốn']
39215916


In [9]:
alphabet = ['\x00', ' ', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'á', 'à', 'ả', 'ã', 'ạ', 'â', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ă', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ', 'ó', 'ò', 'ỏ', 'õ', 'ọ', 'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ', 'é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ê', 'ế', 'ề', 'ể', 'ễ', 'ệ', 'ú', 'ù', 'ủ', 'ũ', 'ụ', 'ư', 'ứ', 'ừ', 'ử', 'ữ', 'ự', 'í', 'ì', 'ỉ', 'ĩ', 'ị', 'ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ', 'đ', 'Á', 'À', 'Ả', 'Ã', 'Ạ', 'Â', 'Ấ', 'Ầ', 'Ẩ', 'Ẫ', 'Ậ', 'Ă', 'Ắ', 'Ằ', 'Ẳ', 'Ẵ', 'Ặ', 'Ó', 'Ò', 'Ỏ', 'Õ', 'Ọ', 'Ô', 'Ố', 'Ồ', 'Ổ', 'Ỗ', 'Ộ', 'Ơ', 'Ớ', 'Ờ', 'Ở', 'Ỡ', 'Ợ', 'É', 'È', 'Ẻ', 'Ẽ', 'Ẹ', 'Ê', 'Ế', 'Ề', 'Ể', 'Ễ', 'Ệ', 'Ú', 'Ù', 'Ủ', 'Ũ', 'Ụ', 'Ư', 'Ứ', 'Ừ', 'Ử', 'Ữ', 'Ự', 'Í', 'Ì', 'Ỉ', 'Ĩ', 'Ị', 'Ý', 'Ỳ', 'Ỷ', 'Ỹ', 'Ỵ', 'Đ']
print(alphabet)
print(len(alphabet))

['\x00', ' ', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'á', 'à', 'ả', 'ã', 'ạ', 'â', 'ấ', 'ầ', 'ẩ', 'ẫ', 'ậ', 'ă', 'ắ', 'ằ', 'ẳ', 'ẵ', 'ặ', 'ó', 'ò', 'ỏ', 'õ', 'ọ', 'ô', 'ố', 'ồ', 'ổ', 'ỗ', 'ộ', 'ơ', 'ớ', 'ờ', 'ở', 'ỡ', 'ợ', 'é', 'è', 'ẻ', 'ẽ', 'ẹ', 'ê', 'ế', 'ề', 'ể', 'ễ', 'ệ', 'ú', 'ù', 'ủ', 'ũ', 'ụ', 'ư', 'ứ', 'ừ', 'ử', 'ữ', 'ự', 'í', 'ì', 'ỉ', 'ĩ', 'ị', 'ý', 'ỳ', 'ỷ', 'ỹ', 'ỵ', 'đ', 'Á', 'À', 'Ả', 'Ã', 'Ạ', 'Â', 'Ấ', 'Ầ', 'Ẩ', 'Ẫ', 'Ậ', 'Ă', 'Ắ', 'Ằ', 'Ẳ', 'Ẵ', 'Ặ', 'Ó', 'Ò', 'Ỏ', 'Õ', 'Ọ', 'Ô', 'Ố', 'Ồ', 'Ổ', 'Ỗ', 'Ộ', 'Ơ', 'Ớ', 'Ờ', 'Ở', 'Ỡ', 'Ợ', 'É', 'È', 'Ẻ', 'Ẽ', 'Ẹ', 'Ê', 'Ế', 'Ề', 'Ể', 'Ễ', 'Ệ', 'Ú', 'Ù', 'Ủ', 'Ũ', 'Ụ', 'Ư', 'Ứ', 'Ừ', 'Ử', 'Ữ', 'Ự', 'Í', 'Ì', 'Ỉ', 'Ĩ', 'Ị', 'Ý', 'Ỳ', 'Ỷ', 'Ỹ', 'Ỵ', 'Đ']
1

In [10]:
MAXLEN=40
BATCH_SIZE=512

In [11]:
def encoder_data(text, maxlen=MAXLEN):
        text = "\x00" + text
        x = np.zeros((maxlen, len(alphabet)))
        for i, c in enumerate(text[:maxlen]):
            x[i, alphabet.index(c)] = 1
        if i < maxlen - 1:
          for j in range(i+1, maxlen):
            x[j, 0] = 1
        return x
      
def decoder_data(x):
    x = x.argmax(axis=-1)
    return ''.join(alphabet[i] for i in x)
encoder_data('Tôi tên bảo')


array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [12]:
!pip install phased_lstm_keras 

from keras.models import Sequential
from tensorflow.keras import layers
from keras.callbacks import Callback, ModelCheckpoint
from keras.layers import Activation, TimeDistributed, Bidirectional,LSTM,Dense
from keras.optimizers import Adam
encoder=LSTM(256,input_shape=(MAXLEN, len(alphabet)), return_sequences=True)
decoder=Bidirectional(LSTM(256, return_sequences=True, dropout=0.2))
model=Sequential()
model.add(encoder)
model.add(decoder)
model.add(TimeDistributed(Dense(256)))
model.add(Activation('relu'))
model.add(TimeDistributed(Dense(len(alphabet))))
model.add(Activation('softmax'))

In [13]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 40, 256)           466944    
_________________________________________________________________
bidirectional (Bidirectional (None, 40, 512)           1050624   
_________________________________________________________________
time_distributed (TimeDistri (None, 40, 256)           131328    
_________________________________________________________________
activation (Activation)      (None, 40, 256)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 40, 199)           51143     
_________________________________________________________________
activation_1 (Activation)    (None, 40, 199)           0         
Total params: 1,700,039
Trainable params: 1,700,039
Non-trainable params: 0
______________________________________________

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [14]:
from sklearn.model_selection import train_test_split
train_data, valid_data = train_test_split(list_ngrams[0:7000000], test_size=0.15, random_state=42)
def generate_data(data, batch_size):
    cur_index = 0
    while True:
        
        x, y = [], []
        for i in range(batch_size):  
            y.append(encoder_data(data[cur_index]))
            x.append(encoder_data(add_noise(data[cur_index],0.94,0.985)))
            cur_index += 1
            
            if cur_index > len(data)-1:
                cur_index = 0
        
        yield np.array(x), np.array(y)

In [ ]:
import os

train_generator = generate_data(train_data, batch_size=BATCH_SIZE)
validation_generator= generate_data(valid_data, batch_size=BATCH_SIZE)

checkpointer = ModelCheckpoint(filepath=os.path.join('gdrive/MyDrive/Spell Correction/Spell_correction.h5'), save_best_only=True, verbose=1)

model.fit(train_generator, steps_per_epoch=len(train_data)//BATCH_SIZE, epochs=10,
                    validation_data=validation_generator, validation_steps=len(valid_data)//BATCH_SIZE,callbacks=[checkpointer])

Epoch 1/10
  803/11621 [=>............................] - ETA: 13:18:13 - loss: 1.4638 - accuracy: 0.6894